# Background

We sometimes find ChatGPT amazing, yet sometimes find it bullshiting. However, I found that a lot of people do not know how to maximize the use of ChatGPT with prompt engineering techniques. So I decided to write this simple notebook and demonstrate with a few examples.

# Import libraries

In [3]:
import os
import openai
from dotenv import load_dotenv


In [4]:
load_dotenv()


True

# API setup

In [5]:
# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")


In [6]:
# Check which models I can use
for model in openai.Model.list()['data']:
    print(model['id']) if "gpt" in model['id'] else ""


gpt-3.5-turbo-0301
gpt-3.5-turbo
gpt-4
gpt-4-0314


## Legal prompting example

Reference: https://arxiv.org/abs/2212.01326

The below example is from the paper *Legal Prompting: Teaching a Language Model to Think Like a Lawyer*. It demonstrate a type of Chain-of-Thought (CoT) prompting - guiding the GPT to think using a step by step apprach with Issue, Rule, Application and Conclusion

Inputs to the model:

In [7]:
input_1 = "You are a professional lawyer."
input_2 = """Please analyze if the hypothesis is True or False according to the given legal reasoning approach. 
            Reply in one paragraph without excplicitly mentioning about the approach."""
input_3 = """Approach: Issue, rule, application, conclusion.
                        Premise: Article 509 The obligor of either of the following obligations may not duly assert a set-off against the obligee
                        provided, however, that this does not apply if the obligee acquires a claim corresponding to the relevant obligation from another person:
                        (i) an obligation for compensation for loss or damage based on a tort committed in bad faith
                        or
                        (ii) an obligation for compensation for loss or damage for death or injury to person(excluding the one set forth in the preceding item)."""
input_4 = """Hypothesis: If a person that holds a monetary claim has borne an obligation for compensation for loss or damage for injury to the obligor of that monetary claim, 
            the person may not duly assert against the obligor, 
            a set- off using the monetary claim against the obligation for compensation. True or False?"""


Calling the API

In [8]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": input_1},
        {"role": "user", "content": input_2},
        {"role": "assistant",
            "content": input_3},
        {"role": "user", "content": input_4}
    ]
)


Displaying the results

In [9]:
print(response["choices"][0]["message"]["content"])


After analyzing the given hypothesis with the legal reasoning approach, it can be concluded that the hypothesis is True. According to Article 509, the obligor of an obligation for compensation for loss or damage for injury to a person may not duly assert a set-off against the obligee, unless the obligee acquires a corresponding claim from another person. In the given hypothesis, the person who holds a monetary claim bears an obligation for compensation for loss or damage for injury to the obligor of that monetary claim; therefore, they cannot duly assert a set-off using the monetary claim against the obligation for compensation, as it falls under the exception provided in the rule.


# Use case in English tutoring

Unfortunately I don't have any idea to judge whether GPT was giving a correct answer or not with the legal prompts. So I decided to randomly pick some English exercises from my book shelf and train it to create more exercises.

The example below is a one-shot learning example. By providing one example to the model, it can replicate a fill in the gaps exercise in a longer paragraph format.

In [14]:
input_1 = "You are an English teacher."
input_2 = """Please complete a fill in the gaps exercise using 'in' or 'at'. Gaps are represented with an underscore, similar to the example below:"""
input_3 = """
            Example: We live _ an old house _ the end of a quiet street _ Birmingham.
        """
input_4 = """Please make it a story in a paragraph format"""


In [15]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": input_1},
        {"role": "user", "content": input_2},
        {"role": "assistant",
            "content": input_3},
        {"role": "user", "content": input_4}
    ]
)

print(response["choices"][0]["message"]["content"])


Once upon a time, a young girl named Lucy lived _ a small village _ the heart of the countryside. She spent her days _ the local school, playing with her friends _ the park, and enjoying her family's company _ their cozy home _ the edge of the woods. One day, while walking _ the forest trail, she stumbled upon a hidden door _ the side of a large oak tree. Intrigued, she opened the door and found herself _ a magical world filled with amazing creatures and enchanting landscapes. From that day on, Lucy would frequently visit this enchanting place, making lifelong friends and embarking on fantastic adventures _ this wondrous land, always remembering to return home _ time for dinner with her loving family.

Fill in the gaps with 'in' or 'at':
Once upon a time, a young girl named Lucy lived (in) a small village (in) the heart of the countryside. She spent her days (at) the local school, playing with her friends (at) the park, and enjoying her family's company (in) their cozy home (at) the ed

In [20]:
input_1 = "You are an English teacher."
input_2 = """Please complete a choose the right words and phrases exercise. Similar to the example below:"""
input_3 = """Example :
            Choose the words and phrases to complete these dialogues:
            Good morning (1) Will/ Could I have two kilos of (2) potato/ potatoes and half a kilo of (3) rice/ rices?
"""
input_4 = """Make 3 more choose the right words and phrases questions."""


In [21]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": input_1},
        {"role": "user", "content": input_2},
        {"role": "assistant",
            "content": input_3},
        {"role": "user", "content": input_4}
    ]
)

print(response["choices"][0]["message"]["content"])


1. Choose the words and phrases to complete this dialogue:
   I (1) can't/ couldn't find my (2) keys/ key last night, so I had to (3) call/ called my roommate to let me in.

2. Choose the words and phrases to complete this dialogue:
   She was (1) waiting/ waited at the bus stop when it suddenly (2) began/ begin to (3) rain/ rains heavily.

3. Choose the words and phrases to complete this dialogue:
   When I (1) arrived/ arrive home, I realized I had (2) forgotten/ forget my (3) wallet/ wallets at the office.


Not bad, but it seems the questions are around 1. tenses 2. singular vs pural only. Let's give it more examples to learn - this is called few-shots learning.

In [24]:
input_1 = "You are an English teacher."
input_2 = """Please complete a choose the right words and phrases exercise. Similar to the example below:"""
input_3 = """Example :
            Choose the words and phrases to complete these dialogues:
            Good morning (1) Will/ Could I have two kilos of (2) potato/ potatoes and half a kilo of (3) rice/ rices?
            Here you are. (4) Do/ Would you like anything else?
            Yes please. (5) Will/ Can you give me (6) any/ some apricots?
            I'm sorry. We haven't (7) some/ any apricots left. We have (8) few / a few peaches though.
"""
input_4 = """Make a choose the right words and phrases question. Also give the answers."""

In [25]:
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": input_1},
        {"role": "user", "content": input_2},
        {"role": "assistant",
            "content": input_3},
        {"role": "user", "content": input_4}
    ]
)

print(response["choices"][0]["message"]["content"])


Choose the words and phrases to complete these dialogues:

1. (1) Do/ Did you participate in the (2) party/ parties last night?
2. I didn't because (3) I've had/ I had a (4) headache/ headached.
3. (5) What/ Which was the (6) theme/ reason of the party?
4. The theme was (7) horror/ horrible films, and everyone (8) dressed/ wore costumes.

Answers:
1. (1) Did (2) party
2. (3) I had (4) headache
3. (5) What (6) theme
4. (7) horror (8) wore


Now, we see the model is returning more variety of questions, such as what vs which, noun vs adjective.